In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from datasets import Dataset

In [3]:
df = pd.read_csv('estonianvalence.csv', encoding="utf-8")

In [4]:
df = df[["valence", "paragraph"]]
df = df[df["valence"] != "vastuoluline"]
df = df.rename(columns={"valence": "label", "paragraph": "text"})
df = df.reset_index(drop=True)

In [5]:
df

,label,text
0,negatiivne,Enam kui kümme aastat tagasi tegutses huumoris...
1,positiivne,Isiklikult kohtasin natukegi Kukekese moodi po...
2,negatiivne,Kummaline on nüüd äkki lugeda politsei ja sise...
3,negatiivne,"Küsimus pole mitte leebetes karistustes, vaid ..."
4,negatiivne,Ajakirjanikuna ei saa ma politseile soovitada ...
...,...,...
3531,negatiivne,Hispaania peaminister Mariano Rajoy on Euroopa...
3532,negatiivne,"Hispaania keskpank on hoiatanud, et riik ei pr..."
3533,neutraalne,Gruusia parlamendivalimised võitnud koalitsioo...
3534,neutraalne,Gruusia meedia on juba spekuleerinud tulevase ...


In [7]:
sentiment_numbers = {'negatiivne': 0, 'neutraalne': 1, 'positiivne': 2}
df['label'] = df['label'].replace(sentiment_numbers)
df

,label,text
0,0,Enam kui kümme aastat tagasi tegutses huumoris...
1,2,Isiklikult kohtasin natukegi Kukekese moodi po...
2,0,Kummaline on nüüd äkki lugeda politsei ja sise...
3,0,"Küsimus pole mitte leebetes karistustes, vaid ..."
4,0,Ajakirjanikuna ei saa ma politseile soovitada ...
...,...,...
3531,0,Hispaania peaminister Mariano Rajoy on Euroopa...
3532,0,"Hispaania keskpank on hoiatanud, et riik ei pr..."
3533,1,Gruusia parlamendivalimised võitnud koalitsioo...
3534,1,Gruusia meedia on juba spekuleerinud tulevase ...


In [8]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)

In [9]:
small_train_dataset = dataset["train"]
small_test_dataset = dataset["test"]

In [10]:
small_train_dataset[0]

{'label': 0,
 'text': 'Jõhkra käitumisega nooruk on varem kriminaalkorras karistamata. Teda on karistatud väärteo, tubakaseaduse rikkumise eest 12 euro suuruse trahviga mullu mais. Juunis sai ta samuti 12 eurot trahvi.'}

In [11]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("EMBEDDIA/est-roberta")
#model = AutoModelForMaskedLM.from_pretrained("EMBEDDIA/est-roberta", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("EMBEDDIA/est-roberta", num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/est-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weig

In [12]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2828 [00:00<?, ? examples/s]

Map:   0%|          | 0/708 [00:00<?, ? examples/s]

In [13]:
print("Batch size of tokenized_train dataset:", len(tokenized_train))
print("Batch size of tokenized_test dataset:", len(tokenized_test))

Batch size of tokenized_train dataset: 2828
Batch size of tokenized_test dataset: 708


In [14]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]

    return {"accuracy": accuracy, "f1": f1}

In [16]:
#!pip install transformers[torch]

In [17]:
import torch
torch.cuda.is_available()

True

In [18]:
#!pip uninstall transformers accelerate
#!pip install transformers accelerate

In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

Step,Training Loss
500,0.597100
1000,0.193000
1500,0.055300


TrainOutput(global_step=1770, training_loss=0.24334039957509876, metrics={'train_runtime': 782.3692, 'train_samples_per_second': 36.147, 'train_steps_per_second': 2.262, 'total_flos': 1699067228420016.0, 'train_loss': 0.24334039957509876, 'epoch': 10.0})

In [21]:
trainer.evaluate()

<ipython-input-15-6f0f0b6fb5c5>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'eval_loss': 1.2142927646636963,
 'eval_accuracy': 0.7838983050847458,
 'eval_f1': 0.7822232365313707,
 'eval_runtime': 7.7963,
 'eval_samples_per_second': 90.812,
 'eval_steps_per_second': 5.772,
 'epoch': 10.0}

In [22]:
model.save_pretrained('/content/drive/MyDrive/MAKATÖÖ/roBertaSentiment_V2')